In [1]:
import tensorflow as tf

In [2]:
# Create example data
data = {
    'Age': 29,
    'Movie': ['The Shawshank Redemption', 'Flight Club'],
    'Movie Ratings': [9.0, 9.7],
    'Suggestion': 'Inception',
    'Suggestion Purchased': 1.0,
    'Purchase Price': 9.99
}

print(data)

{'Age': 29, 'Movie Ratings': [9.0, 9.7], 'Movie': ['The Shawshank Redemption', 'Flight Club'], 'Suggestion': 'Inception', 'Purchase Price': 9.99, 'Suggestion Purchased': 1.0}


In [3]:
# Create the Example
example = tf.train.Example(features=tf.train.Features(feature={
    'Age': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['Age']])),
    'Movie': tf.train.Feature(bytes_list=tf.train.BytesList(value=[m.encode('utf-8') for m in data['Movie']])),
    'Movie Ratings': tf.train.Feature(float_list=tf.train.FloatList(value=data['Movie Ratings'])),
    'Suggestion': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['Suggestion'].encode('utf-8')])),
    'Suggestion Purchased': tf.train.Feature(float_list=tf.train.FloatList(value=[data['Suggestion Purchased']])),
    'Purchase Price': tf.train.Feature(float_list=tf.train.FloatList(value=[data['Purchase Price']]))
}))

print(example)

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 29
      }
    }
  }
  feature {
    key: "Movie"
    value {
      bytes_list {
        value: "The Shawshank Redemption"
        value: "Flight Club"
      }
    }
  }
  feature {
    key: "Movie Ratings"
    value {
      float_list {
        value: 9.0
        value: 9.699999809265137
      }
    }
  }
  feature {
    key: "Purchase Price"
    value {
      float_list {
        value: 9.989999771118164
      }
    }
  }
  feature {
    key: "Suggestion"
    value {
      bytes_list {
        value: "Inception"
      }
    }
  }
  feature {
    key: "Suggestion Purchased"
    value {
      float_list {
        value: 1.0
      }
    }
  }
}



In [4]:
# Write TFrecord file
import os

file_name = '../data'
if not os.path.isdir(file_name):
    os.makedirs(file_name)
    
with tf.python_io.TFRecordWriter(os.path.join(file_name, 'customer_1.tfrecord')) as writer:
    writer.write(example.SerializeToString())

In [5]:
# Read and print data:
sess = tf.InteractiveSession()

In [10]:
# Read TFRecord file
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer([os.path.join(file_name, 'customer_1.tfrecord')])
# filename_queue = tf.data.Dataset.from_tensor_slices([os.path.join(file_name, 'customer_1.tfrecord')])
_, serialized_example = reader.read(filename_queue)

# Define features
read_features = {
    'Age': tf.FixedLenFeature([], dtype=tf.int64),
    'Movie': tf.VarLenFeature(dtype=tf.string),
    'Movie Ratings': tf.VarLenFeature(dtype=tf.float32),
    'Suggestion': tf.FixedLenFeature([], dtype=tf.string),
    'Suggestion Purchased': tf.FixedLenFeature([], dtype=tf.float32),
    'Purchase Price': tf.FixedLenFeature([], dtype=tf.float32)}

# Extract features from serialized data
read_data = tf.parse_single_example(serialized=serialized_example, features=read_features)

# Many tf.train functions use tf.train.QueueRunner,
# so we need to start it before we read
tf.train.start_queue_runners(sess)

# Print features
for name, tensor in read_data.items():
    print('{}: {}'.format(name, tensor.eval()))

Movie: SparseTensorValue(indices=array([[0],
       [1]], dtype=int64), values=array([b'The Shawshank Redemption', b'Flight Club'], dtype=object), dense_shape=array([2], dtype=int64))
Movie Ratings: SparseTensorValue(indices=array([[0],
       [1]], dtype=int64), values=array([9. , 9.7], dtype=float32), dense_shape=array([2], dtype=int64))
Purchase Price: 9.989999771118164
Age: 29
Suggestion: b'Inception'
Suggestion Purchased: 1.0
